In [ ]:
def relative_id_nosort(line_target,line_ref):
    N_target = len(line_target)
    id_target = []
    N_ref = len(line_ref)
    for ip in range(0,N_target):
        val_target = line_target[ip]
        for i in range(0,N_ref-1):
            if (line_ref[i]<=val_target and line_ref[i+1]>val_target):
                id_target.append(i)
                break
        if (len(id_target)<ip+1): 
            id_target.append(-1)    
        if (len(id_target)!=ip+1): 
            print('error')
            print(len(id_target),ip)
    return id_target 


In [ ]:
import h5py
import numpy as np

def read_model_file_horizontal(grid_fname,model_fname,Nstep,stlat,edlat,stlon,edlon,Nlat,Nlon,all_dep):
    R_earth = 6371.0

    fmodel  = h5py.File(model_fname, 'r')
    fgrid   = h5py.File(grid_fname, 'r')

    print(fmodel.keys())

    vel_inv  = np.array(fmodel['model']["vel_inv_%s"%(Nstep)])
    xi_inv  = np.array(fmodel['model']["xi_inv_%s"%(Nstep)])
    eta_inv  = np.array(fmodel['model']["eta_inv_%s"%(Nstep)])

    grid_r = np.array(fgrid["Mesh"]["node_coords_r"])
    grid_t = np.array(fgrid["Mesh"]["node_coords_t"])
    grid_p = np.array(fgrid["Mesh"]["node_coords_p"])
    
    fmodel.close()
    fgrid.close()

    model={}
    dir_lat={};dir_lon={};dir_dep={}
    np_lat=[];np_lon=[];np_dep=[]
    for i in range(len(vel_inv)):
        lat  = '%.8g'%(grid_t[i]); 
        lon  = '%.8g'%(grid_p[i]); 
        dep  = '%.8g'%(R_earth - grid_r[i]); 
        vel = vel_inv[i]
        xi   = xi_inv[i]
        eta  = eta_inv[i]
        sigma = vel_inv[i] 

        model['%s_%s_%s'%(lat,lon,dep)]=(vel,xi,eta,sigma)
        dir_lat[lat] = float(lat)
        dir_lon[lon] = float(lon)
        dir_dep[dep] = float(dep)



    for i in dir_lat:
        np_lat.append(dir_lat[i])
    np_lat.sort()
    for i in dir_lon:
        np_lon.append(dir_lon[i])
    np_lon.sort()
    for i in dir_dep:
        np_dep.append(dir_dep[i])
    np_dep.sort()



    all_lat=np.linspace(stlat,edlat,Nlat,dtype=float)
    all_lon=np.linspace(stlon,edlon,Nlon,dtype=float)



    np_lat_id=[];np_lon_id=[];np_dep_id=[]

    np_lat_id = relative_id_nosort(all_lat,np_lat) 
    np_lon_id = relative_id_nosort(all_lon,np_lon) 
    np_dep_id = relative_id_nosort(all_dep,np_dep) 

    target_dep_value = np.full((len(all_dep),Nlat*Nlon,7),np.nan)    # dep,lat,lon,vel,xi,eta,simga
    for idep in range(0,len(all_dep)):
        dep=all_dep[idep]

        for i in range(0,Nlat):
            lat=all_lat[i]
            
            for j in range(0,Nlon):
                lon=all_lon[j]

                (phy_lat, phy_lon) = (lat,lon)

                #try:

                # 确定了目标点，开始寻找周围八个点
                lat1=np_lat[np_lat_id[i]];lat2=np_lat[np_lat_id[i]+1]
                lon1=np_lon[np_lon_id[j]];lon2=np_lon[np_lon_id[j]+1]
                dep1=np_dep[np_dep_id[idep]];dep2=np_dep[np_dep_id[idep]+1]

                if (not(lat>=lat1 and lat<lat2 and lon>=lon1 and lon<lon2 and dep>=dep1 and dep<dep2)):
                    target_dep_value[idep,i*Nlon+j,:] = (dep,lat,lon,np.nan,np.nan,np.nan,np.nan)
                    # print(lat,lon,dep,"out of region")
                else:
                    # 三个方向比例
                    rdep=(dep-dep1)/(dep2-dep1); rdep=min(rdep,1);rdep=max(0,rdep)
                    rlat=(lat-lat1)/(lat2-lat1); rlat=min(rlat,1);rlat=max(0,rlat)
                    rlon=(lon-lon1)/(lon2-lon1); rlon=min(rlon,1);rlon=max(0,rlon)

                    vel=0
                    vel+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][0]
                    vel+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][0]
                    vel+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][0]
                    vel+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][0]
                    vel+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][0]
                    vel+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][0]
                    vel+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][0]
                    vel+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][0]


                    xi=0
                    xi+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][1]
                    xi+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][1]
                    xi+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][1]
                    xi+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][1]
                    xi+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][1]
                    xi+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][1]
                    xi+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][1]
                    xi+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][1]


                    eta=0
                    eta+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][2]
                    eta+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][2]
                    eta+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][2]
                    eta+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][2]
                    eta+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][2]
                    eta+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][2]
                    eta+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][2]
                    eta+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][2]

                    sigma = 0
                    sigma+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][3]
                    sigma+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][3]
                    sigma+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][3]
                    sigma+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][3]
                    sigma+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][3]
                    sigma+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][3]
                    sigma+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][3]
                    sigma+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][3]

                    target_dep_value[idep,i*Nlon+j,:] = (dep,phy_lat,phy_lon,vel,xi,eta,sigma)
    return target_dep_value


In [ ]:
def ave_model(target_model):
    Ndep  = np.size(target_model,0)
    Ngrid = np.size(target_model,1)
    average_model = np.copy(target_model)    # dep,lat,lon,slowness,xi,eta,vel
    
    for idep in range(Ndep):
        ave_value = 0
        count = 0
        for igrid in range(Ngrid):
            if(np.isnan(target_model[idep,igrid,3])):
                # nan, 
                pass
            else:
                print(target_model[idep,igrid,:])
                print(ave_value,count)
                ave_value = ave_value + target_model[idep,igrid,3]
                count = count + 1
        
        ave_value = ave_value/count

        for igrid in range(Ngrid):
            if(np.isnan(target_model[idep,igrid,3])):
                # nan, 
                pass
            else:
                average_model[idep,igrid,3] = ave_value

    return average_model

In [ ]:
# 函数：read_src_rec_file(fname) 读取到时数据
def read_src_rec_file(fname):
    ev_lat = []
    ev_lon = []
    st_lat = []
    st_lon = []

    ev_names = {}
    st_names = {}

    doc = open(fname,'r')
    doc_input = doc.readlines()
    doc.close()

    cc = 0
    for info in doc_input:
        tmp=info.split()
        if (cc == 0):   # event line
            #  1 2000  1  2 20 28  37.270   38.2843     39.0241  11.00  3.60    8   1725385
            # id_ev   = int(tmp[0])
            ev_name = tmp[12]
            if (not ev_name in ev_names):
                ev_names[ev_name] = 1
                ev_lat.append(float(tmp[7]))
                ev_lon.append(float(tmp[8]))
            ndata = int(tmp[11])
            cc += 1
        else:   # data line
            #  1      1      MYA       38.3261     38.4253   1050.0000  P   52.46   6.630 weight
            if(len(tmp) < 10):
                st_name = tmp[2]
                if (not st_name in st_names):
                    st_names[st_name] = 1
                    st_lat.append(float(tmp[3]))
                    st_lon.append(float(tmp[4]))

            else:   
                phase = tmp[11]
                if (phase.__contains__("cs")):
                    st_name = tmp[2]
                    if (not st_name in st_names):
                        st_names[st_name] = 1
                        st_lat.append(float(tmp[3]))
                        st_lon.append(float(tmp[4]))
                    st_name = tmp[7]
                    if (not st_name in st_names):
                        st_names[st_name] = 1
                        st_lat.append(float(tmp[8]))
                        st_lon.append(float(tmp[9]))
                else:
                    st_name = tmp[2]
                    if (not st_name in st_names):
                        st_names[st_name] = 1
                        st_lat.append(float(tmp[3]))
                        st_lon.append(float(tmp[4]))
                    ev_name = tmp[7]
                    if (not ev_name in ev_names):
                        ev_names[ev_name] = 1
                        ev_lat.append(float(tmp[8]))
                        ev_lon.append(float(tmp[9]))

            if (cc == ndata):
                cc = 0
            else:
                cc += 1
    return [ev_lat,ev_lon,st_lat,st_lon]